In [445]:
# Initial imports
import os
import requests
import pandas as pd
import numpy as np
from dotenv import load_dotenv
import alpaca_trade_api as tradeapi
import matplotlib.pyplot as plt
import hvplot.pandas
%matplotlib inline

In [446]:
# Load .env enviroment variables
load_dotenv()



True

In [447]:
# Set Alpaca API key and secret
alpaca_api_key=os.getenv('ALPACA_KEY')
alpaca_secret_key=os.getenv('ALPACA_SECRET_KEY')

# Create the Alpaca API object
alpaca=tradeapi.REST(
    alpaca_api_key,
    alpaca_secret_key,
    api_version='v2'
)

In [448]:
# Format current date as ISO format
start_time=pd.Timestamp('2023-10-30', tz='America/New_York').isoformat()
end_time=pd.Timestamp('2023-10-31', tz='America/New_York').isoformat()

# Set the tickers
tickers = ["AAPL"]

# Set timeframe to "1Minute" for Alpaca API
timeframe = "1Min"

# Get current closing prices for AAPL and SP500
df_tickers=alpaca.get_bars(
    tickers,
    timeframe,
    start=start_time,
    end=end_time
).df

df_tickers

,close,high,low,trade_count,open,volume,vwap,symbol
timestamp,,,,,,,,
2023-10-30 08:00:00+00:00,168.75,168.98,168.75,68,168.9800,1104,168.760054,AAPL
2023-10-30 08:01:00+00:00,168.84,168.90,168.84,43,168.9000,1204,168.811561,AAPL
2023-10-30 08:02:00+00:00,168.88,168.88,168.70,28,168.7000,1170,168.846556,AAPL
2023-10-30 08:03:00+00:00,168.84,168.84,168.60,50,168.6300,3091,168.685086,AAPL
2023-10-30 08:04:00+00:00,168.88,168.88,168.88,20,168.8800,788,168.886307,AAPL
...,...,...,...,...,...,...,...,...
2023-10-30 23:54:00+00:00,169.84,169.84,169.84,16,169.8400,200,169.839475,AAPL
2023-10-30 23:56:00+00:00,169.86,169.87,169.86,15,169.8700,483,169.851791,AAPL
2023-10-30 23:57:00+00:00,169.85,169.87,169.85,19,169.8699,1120,169.861675,AAPL


In [449]:
# Defining variables
shares = 100




In [450]:
# Reorganize the DataFrame
df_tickers.drop(['high','low','trade_count','volume','vwap','symbol','open'], axis=1, inplace=True)

In [451]:
### Establishing control benchmark
df_tickers['return'] = df_tickers['close'].pct_change() + 1
df_tickers['return'].iat[0] = 1
fund = shares*df_tickers['close'][0]
df_tickers['benchmark'] = fund * df_tickers['return'].cumprod()
df_tickers


,close,return,benchmark
timestamp,,,
2023-10-30 08:00:00+00:00,168.75,1.000000,16875.0
2023-10-30 08:01:00+00:00,168.84,1.000533,16884.0
2023-10-30 08:02:00+00:00,168.88,1.000237,16888.0
2023-10-30 08:03:00+00:00,168.84,0.999763,16884.0
2023-10-30 08:04:00+00:00,168.88,1.000237,16888.0
...,...,...,...
2023-10-30 23:54:00+00:00,169.84,1.000353,16984.0
2023-10-30 23:56:00+00:00,169.86,1.000118,16986.0
2023-10-30 23:57:00+00:00,169.85,0.999941,16985.0


In [452]:
### Creating new Dataframe for MACD calculations
df_macd = pd.DataFrame()
df_macd['MACD'] =  df_tickers['close'].ewm(halflife=12).mean() - df_tickers['close'].ewm(halflife=26).mean()
df_macd['Signal'] = df_macd['MACD'].ewm(halflife=9).mean()
df_macd['histogram'] = df_macd['MACD'] - df_macd['Signal']
df_macd

,MACD,Signal,histogram
timestamp,,,
2023-10-30 08:00:00+00:00,0.000000,0.000000,0.000000
2023-10-30 08:01:00+00:00,0.000699,0.000363,0.000336
2023-10-30 08:02:00+00:00,0.001339,0.000714,0.000625
2023-10-30 08:03:00+00:00,0.001161,0.000839,0.000322
2023-10-30 08:04:00+00:00,0.001567,0.001008,0.000559
...,...,...,...
2023-10-30 23:54:00+00:00,-0.046476,-0.043556,-0.002921
2023-10-30 23:56:00+00:00,-0.044257,-0.043608,-0.000649
2023-10-30 23:57:00+00:00,-0.042450,-0.043522,0.001072


In [453]:
### Plotting Close price

Close_plot = df_tickers['close'].hvplot.line(    
    x='timestamp', 
    y='close',
    rot=90,
    frame_width=1000,
    frame_height=500,
    ylabel='AAPL close', 
    xlabel='timestamp',
    title='Closing Price AAPL',
    grid=True,
).opts(
    yformatter='%.0f',
    hover_color="orange",
    gridstyle={'color': 'gray', 'line_width': 1, 'line_style': 'solid'}
)
### Plotting MACD LINE & signal line
MACD_plot = df_macd['MACD'].hvplot.line(    
    x='timestamp', 
    y='MACD',
    rot=90,
    frame_width=1000,
    frame_height=500,
    ylabel='AAPL close', 
    xlabel='timestamp',
    title='MACD Line & Signal Line',
    grid=True,
).opts(
    yformatter='%.0f',
    hover_color="orange",
    gridstyle={'color': 'gray', 'line_width': 1, 'line_style': 'solid'}
)
Signal_plot = df_macd['Signal'].hvplot.line(    
    x='timestamp', 
    y='Signal',
    rot=90,
    frame_width=1000,
    frame_height=500,
    ylabel='AAPL close', 
    xlabel='timestamp',
    title='MACD Line & Signal Line',
    grid=True,
    legend=True,
).opts(
    yformatter='%.0f',
    hover_color="orange",
    gridstyle={'color': 'gray', 'line_width': 1, 'line_style': 'solid'}
)
### Plotting Histogram
Histogram_plot = df_macd['histogram'].hvplot.bar(    
    x='timestamp', 
    y='histogram',
    rot=90,
    frame_width=1000,
    frame_height=500,
    ylabel='MACD-Signal', 
    xlabel='timestamp',
    title='MACD Line - Signal Line',
    grid=True,
    legend=True,
).opts(
    yformatter='%.0f',
    hover_color="orange",
    gridstyle={'color': 'gray', 'line_width': 1, 'line_style': 'solid'}
)


In [454]:
display(Close_plot)
display(MACD_plot * Signal_plot)
Histogram_plot

:Curve   [timestamp]   (close)

:Overlay
   .Curve.MACD   :Curve   [timestamp]   (MACD)
   .Curve.Signal :Curve   [timestamp]   (Signal)

:Bars   [timestamp]   (histogram)

### I HAVE CALCULATED THE MACD LINE THE SIGNAL LINE
### I WILL BE USING A CROSSOVER STRATEGY TO MAKE MONEY ON IT.
Investopedia
Crossover Strategy
A crossover occurs when the signal and MACD line cross each other. The MACD generates a bullish signal when it moves above its own nine-day EMA and sends a sell signal (bearish) when it moves below its nine-day EMA.

In [455]:
# Creating signals
# Finding long or short trades
df_macd["X"]=np.where(df_macd["MACD"]>df_macd["Signal"],1,0) 
df_macd["X"]=df_macd["X"].diff() 
df_macd['close'] = df_tickers['close']
display(df_macd)

# Extracting the timestamps that indicate bullish signals (Signals where we buy) and that indicate bearish signals (signals where we sell)
Xreturns=df_macd[df_macd["X"].isin([1,-1])] 


# Dataframe with trade signals and dates
Xreturns['cashflow'] = -Xreturns['X'] * Xreturns['close'] * shares
cashflow = Xreturns["cashflow"].sum()
display(Xreturns)
display(cashflow)

,MACD,Signal,histogram,X,close
timestamp,,,,,
2023-10-30 08:00:00+00:00,0.000000,0.000000,0.000000,NaN,168.75
2023-10-30 08:01:00+00:00,0.000699,0.000363,0.000336,1.0,168.84
2023-10-30 08:02:00+00:00,0.001339,0.000714,0.000625,0.0,168.88
2023-10-30 08:03:00+00:00,0.001161,0.000839,0.000322,0.0,168.84
2023-10-30 08:04:00+00:00,0.001567,0.001008,0.000559,0.0,168.88
...,...,...,...,...,...
2023-10-30 23:54:00+00:00,-0.046476,-0.043556,-0.002921,0.0,169.84
2023-10-30 23:56:00+00:00,-0.044257,-0.043608,-0.000649,0.0,169.86
2023-10-30 23:57:00+00:00,-0.042450,-0.043522,0.001072,1.0,169.85


c:\Users\Micha\anaconda3\envs\project\lib\site-packages\ipykernel_launcher.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  del sys.path[0]


,MACD,Signal,histogram,X,close,cashflow
timestamp,,,,,,
2023-10-30 08:01:00+00:00,0.000699,0.000363,0.000336,1.0,168.8400,-16884.00
2023-10-30 08:27:00+00:00,0.003453,0.004758,-0.001305,-1.0,168.8500,16885.00
2023-10-30 09:04:00+00:00,0.003145,0.000266,0.002879,1.0,169.0900,-16909.00
2023-10-30 10:19:00+00:00,0.028635,0.030118,-0.001483,-1.0,169.0200,16902.00
2023-10-30 11:01:00+00:00,0.021043,0.019355,0.001688,1.0,169.1800,-16918.00
2023-10-30 11:30:00+00:00,0.029353,0.030492,-0.001140,-1.0,169.1300,16913.00
2023-10-30 11:36:00+00:00,0.030867,0.030284,0.000583,1.0,169.2100,-16921.00
2023-10-30 11:57:00+00:00,0.031866,0.032835,-0.000969,-1.0,169.1000,16910.00
2023-10-30 13:05:00+00:00,-0.116469,-0.119259,0.002790,1.0,168.7392,-16873.92


-28.95999999999185

In [456]:
### Cashflow is being calculated as the sum of all outflows and inflows of cash. The outflows are when we are buying, that inflows are when we are selling.
### The benchmark is being calculated as a result of the initial value of 100 shares defined in the fund variable minus the last value in the df_tickers['benchmark'] column.
print(f"Profit/Loss MACD: {cashflow}\nProfit/Loss Benchmark: {fund - df_tickers['benchmark'][-1]}")

Profit/Loss MACD: -28.95999999999185
Profit/Loss Benchmark: -94.99999999991633
